6.1 멀티 프로세스와 멀티 스레드
- 운영 체제에서 실행되는 각 프로그램은 각각이 별도의 프로세스이다.
- 각 프로세스에는 하나 이상의 스레드가 있다.
- 멀티프로세스와 멀티스레드라는 두 가지 방법을 사용하면 프로그램의 작업 부하를 분산시킬 수 있다.

1. 멀티프로세스
    - 각 프로세스마다 별도의 메모리 영역을 가지고, 특별한 메커니즘(ex.큐)으로만 통신가능하다.
    - 파이썬에서는 subprocess 모듈을 사용한다.
2. 멀티스레드
    - 단일 프로세스 내의 멀티 스레드는 동일한 메모리에 접근한다.
    - 데이터 공유를 통해서 간단하게 통신하는데, threading 모듈의 처리를 통해서 한번에 한 스레드만 메모리 영역에 접근할 수 있다.(파이썬의 GIL 제한...)
    - 각 프로세스는 독립적인 스택, 힙, 코드, 데이터 영역을 가지지만, 한 프로세스에 속한 스레드들은 스택을 제외하고 다른 메모리 영역은 공유한다.


**동시성 :
    논리적으로 여러작업이 동시에 실행되는 것 처럼 보이는 것

**병렬성 :
    물리적으로 진짜 여러 작업이 동시에 처리되는 것

6.1.1 subprocess 모듈

In [2]:
import subprocess
subprocess.run(["echo", "이것은 subprocess입니다."])

subprocess.run(["sleep", "10"])

CompletedProcess(args=['sleep', '10'], returncode=0)

6.1.2 Threading 모듈

- 멀티 스레드를 사용하려면 threading 모듈을 사용한다.
- 내부적으로 락을 관리하려면 queue모듈을 사용한다.
- 큐에 의존하면 자원의 접근을 직렬화할 수 있고, 즉 한 번에 하나의 스레드만 데이터에 접근할 수 있게 한다는 말이다.

In [17]:
import queue
import threading

q = queue.Queue()

def worker(num):
    while True:
        item = q.get()              #q라는 큐에서 하나씩 가져와서 item이라는 변수에 지정하겠다.
        if item is None :           #못 가져오면 break 하겟다
            break

        #작업을 처리한다.
        print("스레드 {0} : 처리 완료 {1}".format(num+1, item))
        q.task_done()

if __name__ == "__main__":
    num_worker_threads = 5
    threads = []
    for i in range(num_worker_threads):
        t = threading.Thread(target=worker, args = (i,))
        t.start()
        threads.append(t)
    
    for item in range(20):
        q.put(item)

    #모든 작업이 끝날 때까지 대기한다(block).
    q.join()

    #워커 스레드를 종료한다(stop).
    for i in range(num_worker_threads):
        q.put(None)
    for t in threads:
        t.join()

스레드 1 : 처리 완료 0
스레드 1 : 처리 완료 1
스레드 1 : 처리 완료 2스레드 2 : 처리 완료 3
스레드 2 : 처리 완료 4
스레드 2 : 처리 완료 5
스레드 2 : 처리 완료 6
스레드 2 : 처리 완료 7
스레드 2 : 처리 완료 8
스레드 2 : 처리 완료 9
스레드 2 : 처리 완료 10
스레드 2 : 처리 완료 11
스레드 2 : 처리 완료 12
스레드 2 : 처리 완료 13
스레드 2 : 처리 완료 14
스레드 2 : 처리 완료 15
스레드 2 : 처리 완료 16
스레드 2 : 처리 완료 17
스레드 2 : 처리 완료 18
스레드 2 : 처리 완료 19



6.1.3 뮤텍스와 세마포어
- 뮤텍스는 락과 같고, 공유 리소스에 한 번에 하나의 스레드만 접근할 수 있도록 하는 상호 배제 동시성 제어 정책을 강제하기위해 설계되었다.

- 뮤텍스가 양수일 때부터 스레드는 일을 시작 할 수 있음
- 뮤텍스가 1부터 시작 -> 어떤 스레드가 일을 시작할 때 뮤텍스 값을 1 감소시킨다(락한다는 의미) -> 일을 끝내면 뮤텍스 값을 1 증가시킨다(언락한다는 의미)


In [10]:
from threading import Thread, Lock
import threading

def worker(mutex, data, thread_safe):
    if thread_safe:
        mutex.acquire()
    try:
        print("스레드 {0} : {1}\n".format(threading.get_ident(), data))
    finally :
        if thread_safe:
            mutex.release()

if __name__ == "__main__":
    threads = []
    thread_safe = False
    mutex = Lock()

    for i in range(20):
        t = Thread(target = worker, args=(mutex, i, thread_safe))
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

스레드 140149829121792 : 0

스레드 140149871085312 : 1

스레드 140149871085312 : 2
스레드 140149829121792 : 3


스레드 140149829121792 : 4
스레드 140149871085312 : 5


스레드 140149871085312 : 6

스레드 140149871085312 : 7

스레드 140149829121792 : 8

스레드 140149871085312 : 9

스레드 140149829121792 : 10

스레드 140149829121792 : 11
스레드 140149871085312 : 12

스레드 140149829121792 : 13


스레드 140149829121792 : 14

스레드 140149871085312 : 15
스레드 140149829121792 : 16


스레드 140149837514496 : 17

스레드 140149829121792 : 18

스레드 140149829121792 : 19



In [11]:
#thread_safe 변수를 True로 지정.. -> mutex.acquire가 실행됨

from threading import Thread, Lock
import threading

def worker(mutex, data, thread_safe):
    if thread_safe:
        mutex.acquire()
    try:
        print("스레드 {0} : {1}\n".format(threading.get_ident(), data))
    finally :
        if thread_safe:
            mutex.release()

if __name__ == "__main__":
    threads = []
    thread_safe = True
    mutex = Lock()

    for i in range(20):
        t = Thread(target = worker, args=(mutex, i, thread_safe))
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

스레드 140149829121792 : 0

스레드 140149829121792 : 1

스레드 140149871085312 : 2

스레드 140149871085312 : 3

스레드 140149829121792 : 4

스레드 140149829121792 : 5

스레드 140149871085312 : 6

스레드 140149871085312 : 7

스레드 140149871085312 : 8

스레드 140149829121792 : 9

스레드 140149871085312 : 10

스레드 140149871085312 : 11

스레드 140149829121792 : 12

스레드 140149829121792 : 13

스레드 140149829121792 : 14

스레드 140149829121792 : 15

스레드 140149871085312 : 16

스레드 140149837514496 : 17

스레드 140149862692608 : 18

스레드 140149871085312 : 19



- 세마포어는 뮤텍스보다 더 일반적인 개념..
- 세마포어는 1보다 더 큰 수로 시작할 수 있다.
- 즉, 세마포어 값은 곧 한 번에 자원에 접근할 수 있는 스레드의 수다(스레드가 접근할 때마다 값을 1 감소시키기 때문에)

In [22]:
import threading
import time

class ThreadPool(object):
    def __init__(self):
        self.active = []
        self.lock = threading.Lock()
    
    def acquire(self, name):        #name 이라는 걸 획득했다는 걸 알려주고, 그 결과를 출력
        with self.lock:
            self.active.append(name)
            print("획득 : {0} | 스레드 풀 : {1}".format(name, self.active))
    
    def release(self, name):        #name 이라는 걸 반환했다는 걸 알려주고, 그 결과를 출력
        with self.lock:
            self.active.remove(name)
            print("반환 : {0} | 스레드 풀 : {1}".format(name, self.active))

def worker(semaphore, pool):
    with semaphore:
        name = threading.currentThread().getName()
        pool.acquire(name)
        time.sleep(1)
        pool.release(name)

if __name__ == "__main__":
    threads = []
    pool = ThreadPool()
    semaphore = threading.Semaphore(3)      #세마포어를 3으로 지정
    for i in range(10):
        t = threading.Thread(target = worker, name="스레드 "+ str(i), args=(semaphore, pool))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    
    print(threads)

획득 : 스레드 0 | 스레드 풀 : ['스레드 0']
획득 : 스레드 1 | 스레드 풀 : ['스레드 0', '스레드 1']
획득 : 스레드 2 | 스레드 풀 : ['스레드 0', '스레드 1', '스레드 2']
반환 : 스레드 0 | 스레드 풀 : ['스레드 1', '스레드 2']
반환 : 스레드 2 | 스레드 풀 : ['스레드 1']
획득 : 스레드 3 | 스레드 풀 : ['스레드 1', '스레드 3']
획득 : 스레드 4 | 스레드 풀 : ['스레드 1', '스레드 3', '스레드 4']
반환 : 스레드 1 | 스레드 풀 : ['스레드 3', '스레드 4']
획득 : 스레드 5 | 스레드 풀 : ['스레드 3', '스레드 4', '스레드 5']
반환 : 스레드 3 | 스레드 풀 : ['스레드 4', '스레드 5']
획득 : 스레드 6 | 스레드 풀 : ['스레드 4', '스레드 5', '스레드 6']
반환 : 스레드 4 | 스레드 풀 : ['스레드 5', '스레드 6']
반환 : 스레드 5 | 스레드 풀 : ['스레드 6']
획득 : 스레드 8 | 스레드 풀 : ['스레드 6', '스레드 8']
획득 : 스레드 7 | 스레드 풀 : ['스레드 6', '스레드 8', '스레드 7']
반환 : 스레드 6 | 스레드 풀 : ['스레드 8', '스레드 7']
획득 : 스레드 9 | 스레드 풀 : ['스레드 8', '스레드 7', '스레드 9']
반환 : 스레드 8 | 스레드 풀 : ['스레드 7', '스레드 9']
반환 : 스레드 7 | 스레드 풀 : ['스레드 9']
반환 : 스레드 9 | 스레드 풀 : []
[<Thread(스레드 0, stopped 140149778765568)>, <Thread(스레드 1, stopped 140149787158272)>, <Thread(스레드 2, stopped 140149828597504)>, <Thread(스레드 3, stopped 140149836990208)>, <Thread(스레드 4, stopped 140149

6.1.4 데드락과 스핀락

**데드락
- 두 개 이상의 프로세스나 스레드가 서로 상대방의 작업이 끝나기만을 기다리고 있기 때문에 겨로가적으로 아무것도 완료되지 못하는 상태.
- 이 네가지를 충족하면 데드락이 발생함.
   1. 상호 배제 : 자원은 한번에 한 프로세스(스레드)만 사용할 수 있다.
   2. 점유와 대기 : 한 프로세스가 자원을 갖고있으면서 다른 프로세스가 자원을 버리는 거를 기다리고 있는 상태
   3. 비선점 : 다른 프로세스가 이미 점유한 자원을 강제로 뺏어오지 못한다.
   4. 순환 대기 : 서로가 서로의 점유자원을 대기하고 있는 상태.

**스핀락
- 바쁜대기(특정 공유자원에 대해 두개 이상의 프로세스나 스레드가 그 이용 권한을 획득하려고 대기하는 현상)의 한 형태이다.

6.1.5 스레딩에 대한 구글 파이썬 스타일 가이드

In [16]:
import threading

def consumer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 대기".format(name))
        cond.wait()
        print("{0} 자원 소비".format(name))

def producer(cond):
    name = threading.currentThread().getName()
    print("{0} 시작".format(name))
    with cond:
        print("{0} 자원 생산 후 모든 소비자에게 알림".format(name))
        cond.notifyAll()

if __name__ == "__main__" :
    condition = threading.Condition()
    consumer1 = threading.Thread(name = "소비자1", target = consumer, args = (condition,))          #스레드 이름이 소비자1
    consumer2 = threading.Thread(name = "소비자2", target = consumer, args = (condition,))
    producer = threading.Thread(name = "생산자", target = producer, args = (condition,))

    consumer1.start()
    consumer2.start()
    producer.start()

소비자1 시작
소비자1 대기
소비자2 시작
소비자2 대기
생산자 시작
생산자 자원 생산 후 모든 소비자에게 알림
소비자1 자원 소비
소비자2 자원 소비
